In [ ]:
import pandas as pd
from collections import defaultdict, Counter
import utils
import os
from create_network import get_fraction_color
from tqdm import tqdm

In [ ]:
BASE_PATH = 'graphs/Bundestag/'

In [ ]:
data = pd.read_stata('data/BTVote/voting_behavior_V2_19492021.dta')
data

In [ ]:
eps = data['elecper'].unique()
eps

In [ ]:
ep = 15  # set election period

In [ ]:
DISREGARD_ABSENCE = False

In [ ]:
data_ep = data[data['elecper'] == ep]
data_ep['vote_beh'].unique()

In [ ]:
poll_ids = data_ep['vote_id'].unique().astype(int)

In [ ]:
MD_ids = data_ep['id_de_parliament'].unique().astype(int)

In [ ]:
Parties = data_ep['ppg'].unique()
Parties

In [ ]:
def mdid_to_name(mdid, data_ep):
    md_rows = data_ep[data_ep['id_de_parliament'] == mdid]
    md_row = md_rows.iloc[0]
    return f"{md_row['firstname']} {md_row['lastname']}"

def mdid_to_party(mdid, data_ep):
    md_rows = data_ep[data_ep['id_de_parliament'] == mdid]
    md_row = md_rows.iloc[0]
    return md_row['ppg']

In [ ]:
MD_names = {id: mdid_to_name(id, data_ep) for id in MD_ids}
MD_parties = {id: mdid_to_party(id, data_ep) for id in MD_ids}

In [ ]:
leg_year = utils.ELEC_YEARS[ep-1]

In [ ]:
period = utils.year_to_period(leg_year)
filepath = BASE_PATH+period+'/network'+period+'.net'
os.makedirs(BASE_PATH+period+'/', exist_ok=True)

In [ ]:
vote_options = data_ep['vote_beh'].unique()
SV = {id: defaultdict(int) for id in MD_ids}
for pid in tqdm(poll_ids):
    VC = {vo: [] for vo in vote_options}
    for i, vote in data_ep[data_ep['vote_id'] == pid].iterrows():
        VC[vote['vote_beh']].append(int(vote['id_de_parliament']))
    for voters in VC.values():
        lnv = len(voters)
        for i in range(lnv):
            vo1 = voters[i]
            for j in range(i+1, lnv):
                vo2 = voters[j]
                SV[vo1][vo2] += 1
                SV[vo2][vo1] += 1
E = set()
n_polls = len(poll_ids)
for midx, row in SV.items():
    for midy, cnt in row.items():
        if cnt == 0:
            continue
        if DISREGARD_ABSENCE:
            raise NotImplementedError()
        else:
            weight = round(cnt / n_polls, 3)
        E.add((min(midx, midy), max(midx, midy), weight))
E = list(E)
with open(filepath, 'w') as f:
    MI_LOOKUP = dict()
    f.write('*vertices '+str(len(MD_ids))+'\n')
    for i, mdid in enumerate(MD_ids):
        MI_LOOKUP[mdid] = i+1
        party = MD_parties[mdid]
        line = f'{i+1} "{MD_names[mdid]}" "{party}" "{mdid}" "{get_fraction_color(party)}"\n'
        f.write(line)
    f.write('*edges '+str(len(E))+'\n')
    for midx, midy, weight in E:
        ix, iy  = MI_LOOKUP[midx], MI_LOOKUP[midy]
        f.write(str(ix) + " " + str(iy) + " " + str(weight)+ "\n")


    